In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from mle_toolbox import load_result_logs, visualize
from neuroevobench.metrics import plot_rliable

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


envpool not installed, Atari problems will not work.
envpool not installed, Atari problems will not work.
You need to install brax for Brax tasks:
  pip install git+https://github.com/google/brax.git@main


### Helper to load and normalize results

In [2]:
result_dirs = [
               "../experiments/fmnist_classify/eval/",
               "../experiments/mnist_generate/eval/",
               "../experiments/ant/eval/",
               "../experiments/fetch/eval/",
               "../experiments/addition/eval/",
            #    "../experiments/smnist/eval/",
               "../experiments/asterix/eval/",
               "../experiments/breakout/eval/",
            #    "../experiments/cifar10/eval/"
               ]

strategy_names = ['OpenES', 'PGPE', 'ARS', 'SNES', 'Sep_CMA_ES', 'LES',
                  'SimpleGA', 'SAMR_GA', 'GESMR_GA', 'LGA']
color_palette = sns.color_palette("colorblind", n_colors=len(strategy_names))

In [3]:
def load_and_normalize(log_path, normalize_by="OpenES"):
    meta_log, hyper_log = load_result_logs(log_path)
    min_perf = hyper_log.test_eval_perf.min()
    max_perf = hyper_log.test_eval_perf.max()
    
    results_dict = {}
    for i, s in enumerate(hyper_log.strategy_name):
        results_dict[s] = hyper_log.test_eval_perf[i]

    for k, v in results_dict.items():
        results_dict[k] = (v - min_perf) / (max_perf - min_perf)
    
    norm_perf = float(results_dict[normalize_by])
    for k, v in results_dict.items():
        results_dict[k] = v / norm_perf

    return results_dict

    
all_dicts = []
for dir in result_dirs:
    d = load_and_normalize(dir, normalize_by="OpenES")
    all_dicts.append(d)

dd = defaultdict(list)

for d in all_dicts:
    for key, value in d.items():
        dd[key].append(value)

score_dict = dict(dd)

for k, v in score_dict.items():
    score_dict[k] = np.array(v).reshape(-1, 1)

/Users/rob/anaconda3/envs/mle-toolbox/lib/python3.9/site-packages/numpy/lib/function_base.py:4650: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(
/Users/rob/anaconda3/envs/mle-toolbox/lib/python3.9/site-packages/numpy/lib/function_base.py:4488: RuntimeWarning: invalid value encountered in add
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))


### Load Hyper & Meta-Log

In [4]:
# score_dict = {
#     "OpenAI-ES": np.array([[0], [1], [2], [3]]),
#     "PGPE": np.array([[0], [1], [2], [3]]) + 1,
#     "ARS": np.array([[0], [1], [2], [3]]) + 2,
#     "SNES": np.array([[0], [1], [2], [3]]) + 3,
#     "Sep-CMA-ES": np.array([[0], [1], [2], [3]]) + 4,
#     "LES": np.array([[0], [1], [2], [3]]) + 5,
#     "Gaussian GA": np.array([[0], [1], [2], [3]]) + 6,
#     "SAMR-GA": np.array([[0], [1], [2], [3]]) + 7,
#     "GESMR-GA": np.array([[0], [1], [2], [3]]) + 8,
#     "LGA": np.array([[0], [1], [2], [3]]) + 9,
# }


In [5]:
plot_rliable(score_dict, "iqm", reps=100)

(<Figure size 3600x6000 with 1 Axes>,
 <Axes: title={'center': 'Interquartile Mean (IQM)'}, xlabel='Normalized Performance'>)